In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

In [2]:

driver = webdriver.Chrome()

driver.get('https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile')
driver.implicitly_wait(5)
state="SBSTC"
driver.maximize_window()

route_n=[]
route_l=[]

def scroll(n):
    pause_time=n
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(n)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


def extract_content():
    containers = driver.find_elements(By.CSS_SELECTOR, "div[class='route_link']")
    for container in containers:
        st_names = container.find_element(By.CSS_SELECTOR,'div.route_details a.route')
        route_n.append(st_names.text)
        links = container.find_elements(By.CSS_SELECTOR, 'a')
        for link in links:
            route_l.append(link.get_attribute('href'))

extract_content()


wait =WebDriverWait(driver, 10)
scroll(5)

for page_number in range(0,5):
    try:
        next_page_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, f'//div[@class="DC_117_paginationTable"]//div[@class="DC_117_pageTabs " and text()="{page_number+1}"]')
        ))

        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()

        next_page_button.click()
        time.sleep(5)
        scroll(5)

        extract_content()

        print(page_number)
    
    except Exception as e:
        print("no result")

no result
1
2
3
4


In [14]:
driver = webdriver.Chrome()
driver.maximize_window()
state_name=[]
route_name=[]
route_link=[]
busname=[]
bustype=[]
departing_time=[]
duration=[]
reaching_time=[]
star_rating=[]
price=[]
seats_available=[]

for index, value in enumerate(route_l):
    
    driver.get(value)
    driver.implicitly_wait(5)
    buttons = driver.find_elements(By.CSS_SELECTOR, "div.button > i.p-left-10.icon.icon-down")

# Initialize ActionChains
    actions = ActionChains(driver)

# Iterate through each button
    for button in buttons:
        actions.move_to_element(button).perform()
        wait = WebDriverWait(driver, 10)
        clickable_button = wait.until(EC.element_to_be_clickable(button))
        clickable_button.click()
        time.sleep(2)
        scroll(3)
        time.sleep(2)
    scroll(5)

    containers =driver.find_elements(By.CSS_SELECTOR,"div[class='clearfix bus-item-details']")

    for container in containers:

        state_name.append(state)

        route_link.append(value)
        

        route_name.append(route_n[index])

        bus_name=container.find_element(By.CSS_SELECTOR,'div.travels.lh-24.f-bold.d-color')
        busname.append(bus_name.text)

        bus_type=container.find_element(By.CSS_SELECTOR,'div.bus-type.f-12.m-top-16.l-color.evBus')
        bustype.append(bus_type.text)

        dep_time=container.find_element(By.CSS_SELECTOR, 'div.dp-time.f-19.d-color.f-bold')
        departing_time.append(dep_time.text)

        dur_time=container.find_element(By.CSS_SELECTOR,'div.dur.l-color.lh-24')
        duration.append(dur_time.text)

        reac_time=container.find_element(By.CSS_SELECTOR,'div.bp-time.f-19.d-color.disp-Inline')
        reaching_time.append(reac_time.text)

        rating_elements = container.find_elements(By.CSS_SELECTOR, "div.rating-sec.lh-24 div.lh-18.rating span")
        if rating_elements:
            rating =(rating_elements[0].text)
        else:
            rating = "0.0"
        star_rating.append(rating)

        price_rate=container.find_element(By.CSS_SELECTOR,'div.fare.d-block span.f-19.f-bold')
        price.append(price_rate.text)

        seat_element =container.find_element(By.CSS_SELECTOR,'div.column-eight.w-15.fl').text[0:2]
        seats_available.append(seat_element)
    
   




df = pd.DataFrame({
    'State_Name' :state_name,
    'Route_Name': route_name,
    'Route_Link': route_link,
    'Bus_Name': busname,
    'Bus_Type': bustype,
    'Departing_Time': departing_time,
    'Duration': duration,
    'Reaching_Time': reaching_time,
    'Star_Rating': star_rating,
    'Price': price,
    'Seats_Available': seats_available
})

df.to_csv("South_bengal.csv")